In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_amazon_Electronics_reviews():
    url = "https://www.amazon.com/s?k=Electronics"  
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        product_links = []
        
        for product in soup.find_all("div", class_="sg-col-inner"):
            link = product.find("a", class_="a-link-normal")
            if link:
                product_links.append("https://www.amazon.com" + link['href'])
        
        with open("Electronics_reviews.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Rating", "Comment"])
            
            for link in product_links:
                response = requests.get(link, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, "html.parser")
                    product_title = soup.find("span", class_="a-size-large product-title-word-break").text.strip()
                    
                    review_section = soup.find("div", {"data-hook": "reviews-collapsed"})
                    if review_section:
                        reviews = review_section.find_all("span", {"data-hook": "review-body"})
                        for review in reviews:
                            rating = review.find_previous("i").text.strip()  
                            comment = review.text.strip()  # Get the comment text
                            writer.writerow([rating, comment])
                            
                            # Introduce a delay to avoid overwhelming the server
                            time.sleep(2)  
                else:
                    print(f"Failed to scrape {link}. Status code: {response.status_code}")
                    # Retry the request after waiting for a while
                    time.sleep(10)  
    else:
        print(f"Failed to access the URL. Status code: {response.status_code}")


if __name__ == "__main__":
    scrape_amazon_Electronics()

Failed to access the URL. Status code: 503


The aboce code can be used to scrap data, but amazon and almost all platforms have restricted it, so i got in touch with stanford where they had amazon electronics dataset and they mailed it to me 

### Reading Json File

In [1]:
import json
import os


file_path = os.path.expanduser("~/Desktop/Final Project/Electronics.json")

def stream_json_data(file_path):
    with open(file_path, "r") as json_file:
        for line_number, line in enumerate(json_file, start=1):
            try:
                data = json.loads(line)
                yield data
            except json.JSONDecodeError:
                print(f"Error parsing line {line_number}: {line}")

for index, item in enumerate(stream_json_data(file_path), start=1):
    print(f"Row {index}: {item}")
    if index == 20:
        break

Row 1: {'overall': 5.0, 'verified': True, 'reviewTime': '07 17, 2002', 'reviewerID': 'A1N070NS9CJQ2I', 'asin': '0060009810', 'style': {'Format:': ' Hardcover'}, 'reviewerName': 'Teri Adams', 'reviewText': "This was the first time I read Garcia-Aguilera.  I came upon the name of this book on Live with Regis and Kelly. This book was exactly what I was looking for ... it hit the spot.  I really enjoyed this book because it was well written. Once I started this book it kept me coming back for more. It had culture, family, friendship and romance. I was looking for a little more romance when I picked this book but in the end it turned out to be just right.  I love the main chartachter Margarita (aka Daisy). I've never been to Miami but the way Daisy told the story I certainly felt I'd been there.\nAlso after going through all of Daisy's perils ... I closed the book with a feeling I had grown emotionally as well.", 'summary': 'Hit The Spot!', 'unixReviewTime': 1026864000}
Row 2: {'overall': 5

### Converting it into Excel File

In [5]:
import os
import json
import pandas as pd
from itertools import islice

file_path = os.path.expanduser("~/Desktop/Final Project/Electronics.json")

chunk_size = 1000

json_directory = os.path.dirname(file_path)

output_file_path = os.path.join(json_directory, "Electronics.xlsx")

excel_writer = pd.ExcelWriter(output_file_path, engine="xlsxwriter")

row_counter = 0

with open(file_path, "r") as json_file:
    while True:
        data_chunk = list(islice(json_file, chunk_size))
        if not data_chunk:
            break

        df_chunk = pd.DataFrame([json.loads(line) for line in data_chunk])

        df_chunk.to_excel(excel_writer, sheet_name="Sheet1", index=False, startrow=row_counter)

        row_counter += len(df_chunk)

excel_writer.save()

print("Data saved to Excel file:", output_file_path)

Data saved to Excel file: /Users/0xjoex/Desktop/Final Project/Electronics.xlsx


In [7]:
import pandas as pd
import os

file_path = os.path.expanduser("~/Desktop/Final Project/Electronics.xlsx")

df = pd.read_excel(file_path)

print(df.head())

  overall verified   reviewTime      reviewerID        asin  \
0       5     True  07 17, 2002  A1N070NS9CJQ2I  0060009810   
1       5    False   07 6, 2002  A3P0KRKOBQK1KN  0060009810   
2       5    False   07 3, 2002  A192HO2ICJ75VU  0060009810   
3       4    False  06 30, 2002  A2T278FKFL3BLT  0060009810   
4       5    False  06 28, 2002  A2ZUXVTW8RXBXW  0060009810   

                       style reviewerName  \
0  {'Format:': ' Hardcover'}   Teri Adams   
1  {'Format:': ' Hardcover'}     Willa C.   
2  {'Format:': ' Hardcover'}          Kit   
3  {'Format:': ' Hardcover'}       Andres   
4  {'Format:': ' Hardcover'}         John   

                                          reviewText  \
0  This was the first time I read Garcia-Aguilera...   
1  As with all of Ms. Garcia-Aguilera's books, I ...   
2  I've not read any of Ms Aguilera's works befor...   
3  This romance novel is right up there with the ...   
4  Carolina Garcia Aguilera has done it again.  S...   

             

In [8]:
print("Shape of the dataset:", df.shape)

Shape of the dataset: (1048575, 12)


In [9]:
print("\nBasic Information:")
print(df.info())


Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   overall         1035845 non-null  object
 1   verified        887548 non-null   object
 2   reviewTime      1042266 non-null  object
 3   reviewerID      1048575 non-null  object
 4   asin            1047571 non-null  object
 5   style           902959 non-null   object
 6   reviewerName    1013386 non-null  object
 7   reviewText      1046804 non-null  object
 8   summary         1048484 non-null  object
 9   unixReviewTime  799614 non-null   object
 10  vote            275785 non-null   object
 11  image           72538 non-null    object
dtypes: object(12)
memory usage: 96.0+ MB
None


In [10]:
print("\nSummary Statistics:")
print(df.describe())


Summary Statistics:
        overall verified reviewTime reviewerID        asin  \
count   1035845   887548    1042266    1048575     1047571   
unique    39424      977      39728     638976      267167   
top           5     True       True       True  B000BQ7GW8   
freq     598079   596838     161443      42032       13755   

                                           style     reviewerName  reviewText  \
count                                     902959          1013386     1046804   
unique                                    228626           669843      836949   
top     {'Package Type:': ' Standard Packaging'}  Amazon Customer  Five Stars   
freq                                       33969            26733       38232   

           summary unixReviewTime    vote  \
count      1048484         799614  275785   
unique      447538          72620   14412   
top     Five Stars              2       2   
freq         79448          16755   25023   

                                    

In [11]:
df.head(15)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"07 17, 2002",A1N070NS9CJQ2I,0060009810,{'Format:': ' Hardcover'},Teri Adams,This was the first time I read Garcia-Aguilera...,Hit The Spot!,1026864000,NaN,NaN
1,5,False,"07 6, 2002",A3P0KRKOBQK1KN,0060009810,{'Format:': ' Hardcover'},Willa C.,"As with all of Ms. Garcia-Aguilera's books, I ...",one hot summer is HOT HOT HOT!,1025913600,NaN,NaN
2,5,False,"07 3, 2002",A192HO2ICJ75VU,0060009810,{'Format:': ' Hardcover'},Kit,I've not read any of Ms Aguilera's works befor...,One Hot Summer,1025654400,2,NaN
3,4,False,"06 30, 2002",A2T278FKFL3BLT,0060009810,{'Format:': ' Hardcover'},Andres,This romance novel is right up there with the ...,I love this book!,1025395200,3,NaN
4,5,False,"06 28, 2002",A2ZUXVTW8RXBXW,0060009810,{'Format:': ' Hardcover'},John,Carolina Garcia Aguilera has done it again. S...,One Hot Book,1025222400,NaN,NaN
5,4,False,"06 25, 2002",A21JDG4HA6OLPF,0060009810,{'Format:': ' Hardcover'},Mari,"Not only can she write mysteries,but she sure ...",Carolina does it again!,1024963200,2,NaN
6,5,False,"06 22, 2002",A2NCADLQY962YD,0060009810,{'Format:': ' Hardcover'},Amazon Customer,Once again Garcia-Aguilera has written a book ...,"It's no mystery, Garcia-Aguilera can do romance",1024704000,4,NaN
7,5,False,"01 26, 2006",A2UNA438B0FRA8,0060786817,{'Format:': ' Hardcover'},J. Belfield,The timing is just right for a good book. Actu...,A breath of fresh air for reading,1138233600,NaN,NaN
8,5,False,"01 21, 2006",A2NQQSHSQKKGEX,0060786817,{'Format:': ' Hardcover'},E. D. Davis,Engaging. Dark. Reading the book you could alm...,Well written page turner with fully develped c...,1137801600,4,NaN
9,3,False,"01 18, 2006",A5EXND10WD4PM,0060786817,{'Format:': ' Hardcover'},Mocha Girl,Set amid the backdrop of New York City's imper...,The Title Tells The Story!,1137542400,7,NaN


In [12]:
df = df.drop(columns=['reviewerName', 'vote', 'image'])

In [13]:
df.head(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewText,summary,unixReviewTime
0,5,True,"07 17, 2002",A1N070NS9CJQ2I,0060009810,{'Format:': ' Hardcover'},This was the first time I read Garcia-Aguilera...,Hit The Spot!,1026864000
1,5,False,"07 6, 2002",A3P0KRKOBQK1KN,0060009810,{'Format:': ' Hardcover'},"As with all of Ms. Garcia-Aguilera's books, I ...",one hot summer is HOT HOT HOT!,1025913600
2,5,False,"07 3, 2002",A192HO2ICJ75VU,0060009810,{'Format:': ' Hardcover'},I've not read any of Ms Aguilera's works befor...,One Hot Summer,1025654400
3,4,False,"06 30, 2002",A2T278FKFL3BLT,0060009810,{'Format:': ' Hardcover'},This romance novel is right up there with the ...,I love this book!,1025395200
4,5,False,"06 28, 2002",A2ZUXVTW8RXBXW,0060009810,{'Format:': ' Hardcover'},Carolina Garcia Aguilera has done it again. S...,One Hot Book,1025222400
5,4,False,"06 25, 2002",A21JDG4HA6OLPF,0060009810,{'Format:': ' Hardcover'},"Not only can she write mysteries,but she sure ...",Carolina does it again!,1024963200
6,5,False,"06 22, 2002",A2NCADLQY962YD,0060009810,{'Format:': ' Hardcover'},Once again Garcia-Aguilera has written a book ...,"It's no mystery, Garcia-Aguilera can do romance",1024704000
7,5,False,"01 26, 2006",A2UNA438B0FRA8,0060786817,{'Format:': ' Hardcover'},The timing is just right for a good book. Actu...,A breath of fresh air for reading,1138233600
8,5,False,"01 21, 2006",A2NQQSHSQKKGEX,0060786817,{'Format:': ' Hardcover'},Engaging. Dark. Reading the book you could alm...,Well written page turner with fully develped c...,1137801600
9,3,False,"01 18, 2006",A5EXND10WD4PM,0060786817,{'Format:': ' Hardcover'},Set amid the backdrop of New York City's imper...,The Title Tells The Story!,1137542400


In [14]:
df_review_text = df[['reviewText']]

In [15]:
print(df_review_text.head(15))

                                           reviewText
0   This was the first time I read Garcia-Aguilera...
1   As with all of Ms. Garcia-Aguilera's books, I ...
2   I've not read any of Ms Aguilera's works befor...
3   This romance novel is right up there with the ...
4   Carolina Garcia Aguilera has done it again.  S...
5   Not only can she write mysteries,but she sure ...
6   Once again Garcia-Aguilera has written a book ...
7   The timing is just right for a good book. Actu...
8   Engaging. Dark. Reading the book you could alm...
9   Set amid the backdrop of New York City's imper...
10  A guilty pleasure of a book with characters so...
11  I'd never before read a Lolita Files' book, an...
12  Wow, I was totally blown away by this book.  J...
13  Lolita Files is one of my favorite authors. Sh...
14  This book is...I don't even know what to call ...


In [16]:
pd.set_option('display.max_colwidth', None)
print(df_review_text.head(5))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  reviewText
0  This was the first time I read Garcia-Aguilera.  I came upon the name of this book on Live with Regis and Kelly. This book was exactly what I was looking for ... it hit the spot.  I really enjoyed this book because it was well written. Once I started this book it kept me coming b

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [18]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/0xjoex/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/0xjoex/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
df_review_text = df_review_text.dropna(subset=['reviewText'])

In [24]:
df_review_text['reviewText'] = df_review_text['reviewText'].astype(str) #covert to string

In [27]:
df_review_text = df_review_text.applymap(str)

In [28]:
# Convert text to lowercase
df_review_text['cleaned_review'] = df_review_text['reviewText'].str.lower()

In [29]:
# Remove punctuation and special characters
df_review_text['cleaned_review'] = df_review_text['cleaned_review'].str.replace('[{}]'.format(string.punctuation), '')

/var/folders/yg/08251k5x0193mhv8vcl_lylm0000gn/T/ipykernel_13383/2318078684.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_review_text['cleaned_review'] = df_review_text['cleaned_review'].str.replace('[{}]'.format(string.punctuation), '')


In [30]:
# Remove numbers
df_review_text['cleaned_review'] = df_review_text['cleaned_review'].str.replace(r'\d+', '')

/var/folders/yg/08251k5x0193mhv8vcl_lylm0000gn/T/ipykernel_13383/3379385073.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_review_text['cleaned_review'] = df_review_text['cleaned_review'].str.replace(r'\d+', '')


In [31]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
df_review_text['cleaned_review'] = df_review_text['cleaned_review'].apply(lambda x: ' '.join(word for word in word_tokenize(x) if word not in stop_words))

In [32]:
df_review_text.head(10)

,reviewText,cleaned_review
0,"This was the first time I read Garcia-Aguilera. I came upon the name of this book on Live with Regis and Kelly. This book was exactly what I was looking for ... it hit the spot. I really enjoyed this book because it was well written. Once I started this book it kept me coming back for more. It had culture, family, friendship and romance. I was looking for a little more romance when I picked this book but in the end it turned out to be just right. I love the main chartachter Margarita (aka Daisy). I've never been to Miami but the way Daisy told the story I certainly felt I'd been there.\nAlso after going through all of Daisy's perils ... I closed the book with a feeling I had grown emotionally as well.",first time read garciaaguilera came upon name book live regis kelly book exactly looking hit spot really enjoyed book well written started book kept coming back culture family friendship romance looking little romance picked book end turned right love main chartachter margarita aka daisy ive never miami way daisy told story certainly felt id also going daisys perils closed book feeling grown emotionally well
1,"As with all of Ms. Garcia-Aguilera's books, I think this is a MUST READ, impossible to put down. Successful deviation from past Lupe Solano series-captures the very essence of the excitement, local color and diverse fabric of MIAMI. Sensual and culturally enlightened!",ms garciaaguileras books think must read impossible put successful deviation past lupe solano seriescaptures essence excitement local color diverse fabric miami sensual culturally enlightened
2,"I've not read any of Ms Aguilera's works before, but after having just finished One Hot Summer I'm going to check out the Lupe Solano series I've heard so much about. One Hot Summer is sooo steamy! Made me want to move to Miami! Couldn't put the book down.",ive read ms aguileras works finished one hot summer im going check lupe solano series ive heard much one hot summer sooo steamy made want move miami couldnt put book
3,"This romance novel is right up there with the rest of her amazing mystery novels. Being a guy, I was a little hesitant about reading a romance novel but I just had to give this book a shot because I have been such a huge fan of Garcia-Aguilera's books. And to be honest, I absolutely loved this book. I love the way she presents funky Miami and its crazy Cubans in not just this book but all her books. Garcia-Aguilera did a superb job with this book, and I can't wait till her next book. You gotta read this book!!!!",romance novel right rest amazing mystery novels guy little hesitant reading romance novel give book shot huge fan garciaaguileras books honest absolutely loved book love way presents funky miami crazy cubans book books garciaaguilera superb job book cant wait till next book got ta read book
4,Carolina Garcia Aguilera has done it again. She's written another highly enjoyable book and infused it with the right amount of Cuban-American tidbits. My family and I cannot put her books down once we start and this one was not a let down.,carolina garcia aguilera done shes written another highly enjoyable book infused right amount cubanamerican tidbits family put books start one let
5,"Not only can she write mysteries,but she sure can write a love story! This was ""ONE HOT SUMMER"" read that I couldn't put down !!!",write mysteriesbut sure write love story one hot summer read couldnt put
6,"Once again Garcia-Aguilera has written a book that I just can't put down. I have read and love all her mysteries- so I picked up this romance with eager anticipation. I was not disappointed. The main character Margarita is easily likeable, which makes the trials she faces that much more intriguing. Sometimes you want to yell at the book and tell Margarita what she should (and shouldn't) do. But the author is measured in the development of the plot and keeps you turning the pages all the way to the end. Mix that in with an interestin

## Sentiment Analysis 

In [33]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [34]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/0xjoex/nltk_data...


True

In [35]:
sia = SentimentIntensityAnalyzer()

In [36]:
def get_sentiment(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [37]:
df_review_text['sentiment_score'] = df_review_text['cleaned_review'].apply(lambda x: sia.polarity_scores(x)['compound'])
df_review_text['sentiment'] = df_review_text['sentiment_score'].apply(get_sentiment)

In [38]:
df_review_text.head(10)

,reviewText,cleaned_review,sentiment_score,sentiment
0,"This was the first time I read Garcia-Aguilera. I came upon the name of this book on Live with Regis and Kelly. This book was exactly what I was looking for ... it hit the spot. I really enjoyed this book because it was well written. Once I started this book it kept me coming back for more. It had culture, family, friendship and romance. I was looking for a little more romance when I picked this book but in the end it turned out to be just right. I love the main chartachter Margarita (aka Daisy). I've never been to Miami but the way Daisy told the story I certainly felt I'd been there.\nAlso after going through all of Daisy's perils ... I closed the book with a feeling I had grown emotionally as well.",first time read garciaaguilera came upon name book live regis kelly book exactly looking hit spot really enjoyed book well written started book kept coming back culture family friendship romance looking little romance picked book end turned right love main chartachter margarita aka daisy ive never miami way daisy told story certainly felt id also going daisys perils closed book feeling grown emotionally well,0.9764,Positive
1,"As with all of Ms. Garcia-Aguilera's books, I think this is a MUST READ, impossible to put down. Successful deviation from past Lupe Solano series-captures the very essence of the excitement, local color and diverse fabric of MIAMI. Sensual and culturally enlightened!",ms garciaaguileras books think must read impossible put successful deviation past lupe solano seriescaptures essence excitement local color diverse fabric miami sensual culturally enlightened,0.8807,Positive
2,"I've not read any of Ms Aguilera's works before, but after having just finished One Hot Summer I'm going to check out the Lupe Solano series I've heard so much about. One Hot Summer is sooo steamy! Made me want to move to Miami! Couldn't put the book down.",ive read ms aguileras works finished one hot summer im going check lupe solano series ive heard much one hot summer sooo steamy made want move miami couldnt put book,0.0772,Positive
3,"This romance novel is right up there with the rest of her amazing mystery novels. Being a guy, I was a little hesitant about reading a romance novel but I just had to give this book a shot because I have been such a huge fan of Garcia-Aguilera's books. And to be honest, I absolutely loved this book. I love the way she presents funky Miami and its crazy Cubans in not just this book but all her books. Garcia-Aguilera did a superb job with this book, and I can't wait till her next book. You gotta read this book!!!!",romance novel right rest amazing mystery novels guy little hesitant reading romance novel give book shot huge fan garciaaguileras books honest absolutely loved book love way presents funky miami crazy cubans book books garciaaguilera superb job book cant wait till next book got ta read book,0.9858,Positive
4,Carolina Garcia Aguilera has done it again. She's written another highly enjoyable book and infused it with the right amount of Cuban-American tidbits. My family and I cannot put her books down once we start and this one was not a let down.,carolina garcia aguilera done shes written another highly enjoyable book infused right amount cubanamerican tidbits family put books start one let,0.4927,Positive
5,"Not only can she write mysteries,but she sure can write a love story! This was ""ONE HOT SUMMER"" read that I couldn't put down !!!",write mysteriesbut sure write love story one hot summer read couldnt put,0.7579,Positive
6,"Once again Garcia-Aguilera has written a book that I just can't put down. I have read and love all her mysteries- so I picked up this romance with eager anticipation. I was not disappointed. The main character Margarita is easily likeable, which makes the trials she faces that much more intriguing. Sometimes you want to yell at the book and tell Margarita what she should (and shouldn't) do. But the author is meas

## Classification

In [39]:
general_keywords = ['review', 'read', 'book', 'enjoyed', 'written']
care_keywords = ['complaint', 'problem', 'issue', 'unhappy', 'disappointed']
leads_keywords = ['buy', 'purchase', 'product', 'service', 'interested']

In [40]:
def categorize_message(text):
    text = text.lower()
    for keyword in care_keywords:
        if keyword in text:
            return "Care"
    for keyword in leads_keywords:
        if keyword in text:
            return "Leads"
    return "General"

In [41]:
df_review_text['category'] = df_review_text['cleaned_review'].apply(categorize_message)

In [42]:
print(df_review_text[['cleaned_review', 'sentiment', 'category']])

                                                                                                                                                                                                                                                                                                                                                                                                                      cleaned_review  \
0        first time read garciaaguilera came upon name book live regis kelly book exactly looking hit spot really enjoyed book well written started book kept coming back culture family friendship romance looking little romance picked book end turned right love main chartachter margarita aka daisy ive never miami way daisy told story certainly felt id also going daisys perils closed book feeling grown emotionally well   
1                                                                                                                                                       

## Display

In [43]:
pd.set_option('display.max_colwidth', None)

In [44]:
pd.set_option('display.max_colwidth', 50)

In [45]:
print(df_review_text[['cleaned_review', 'sentiment', 'category']])

                                            cleaned_review sentiment category
0        first time read garciaaguilera came upon name ...  Positive  General
1        ms garciaaguileras books think must read impos...  Positive  General
2        ive read ms aguileras works finished one hot s...  Positive  General
3        romance novel right rest amazing mystery novel...  Positive  General
4        carolina garcia aguilera done shes written ano...  Positive  General
...                                                    ...       ...      ...
1048570          mm hiresolution pcs lens filter kit black   Neutral  General
1048571                            low cost filter kit job  Negative  General
1048572                                          filer kit   Neutral  General
1048573                                     best bang buck  Positive  General
1048574                                     great exercise  Positive  General

[1046804 rows x 3 columns]


In [46]:
category_counts = df_review_text['category'].value_counts()
print(category_counts)

General    782626
Leads      168344
Care        95834
Name: category, dtype: int64


In [47]:
total_messages = df_review_text.shape[0]
print("Total number of messages:", total_messages)

Total number of messages: 1046804
